In [1]:
def __return_waypoints_ind():
    d = np.array([
    [ 65, -36],
    [ 56, -21],
    [ 66, -46],
    [ 58, -48],
    [ 67, -29],
    [ 61, -16],
    [ 45, -32],
    [ 71, -43],
    # [ 80, -52],
    # [ 68, -58],
    [ 65, -54],
    [ 48, -20],
    [ 64, -21],
    [ 46, -14]])
    return d

%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
from helpers.highlevel_sceneloader import HighLevelSceneLoader
from predictors.dataset_creator import TFDataSet
import tensorflow as tf
from predictors.dl_trainer import DLTrainer 
from predictors.extended_predictor import extended_predictor 
import matplotlib.pyplot as plt
import pickle
from helpers.graph import Graph
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
from helpers.accuracy_functions import *
import csv
import math
from tqdm import tqdm


''' set some parameters '''
# Model parameters
LSTM_LAYER_SIZE = 64
DENSE_LAYER_SIZE = 128
NUM_LSTM_LAYERS = 2
NUM_DENSE_LAYERS = 2
VARIABLE_INPUT_LENGTH = False

# Dataset
SEQ_IN_LEN = 3
SEQ_OUT_LEN = 8
NOISE_STD = .3
N_REPEATS = 1

BATCH_SIZE = 5
LENGTH_STRIDE = 2


# Training parameters
MAX_EPOCHS = 100
PATIENCE = 5

# For graph
GRAPH_DIST_THRESH = 4

''' get the data '''
ROOT = os.getcwd()

rel_p_img_b = 'helpers/analysed_vars_storage/img_bounds.xml'
rel_p_dests = 'helpers/analysed_vars_storage/destination_locations.xml'
p_img_bounds = os.path.join(ROOT, rel_p_img_b)
p_dest_locs = os.path.join(ROOT, rel_p_dests)

#TODO: older version of OpenTraj needed: "git checkout d249ba6951dd0f54b532fbe2ca6edc46b0d7093f"
opentraj_root = os.path.join(ROOT, 'OpenTraj')
root_datasets = os.path.join(ROOT, 'data/path_data')
sys.path.append(opentraj_root) # add package to pythonpath

scene_data = HighLevelSceneLoader(p_img_bounds, p_dest_locs)
scene_data.load_ind(root_datasets, 7, 17)


''' create the graph instance '''    
interest_points = __return_waypoints_ind()
g = Graph.from_matrices(interest_points, scene_data.destination_matrix, GRAPH_DIST_THRESH, .05)

df_signals = scene_data.df_to_lst_realxy_mats()
g.analyse_multiple_full_signals(df_signals, add_to_trams_mat=True)

''' time to create df datasets '''
extra_features_dict = {
    "all_points": None,
    "all_destinations": None,
    "n_destinations": 5,
    "n_points": 5,
    "n_connected_points_after" : 3
}

# Load data in order to not need to do calculations again
# with open("data/pickle/ds_creation_d/ds_7to17_inputLabels5_6.pickle", 'rb') as handle: #"data/pickle/ds_creation_d/bs1.pickle"
#     my_ds_creation_dict = pickle.load(handle)

my_ds = TFDataSet.init_as_fixed_length(scene_data.traj_dataframe, graph=g, var_in_len=VARIABLE_INPUT_LENGTH, length_stride=LENGTH_STRIDE,
scale_list=["pos_x", "pos_y"], seq_in_length=SEQ_IN_LEN, label_length=SEQ_OUT_LEN,
extra_features_dict=extra_features_dict, noise_std=NOISE_STD, 
n_repeats=N_REPEATS, batch_size=BATCH_SIZE, save_folder = "data/pickle/ds_creation_d/ds_7to17_inputLabels3_8.pickle") #, ds_creation_dict=my_ds_creation_dict


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [ ]:
# ''' time for some model training '''
# # BASIC TRAINER
# my_trainer = DLTrainer(max_epochs=MAX_EPOCHS, patience=PATIENCE)
# my_trainer.LSTM_one_shot_predictor_named_i(my_ds, LSTM_LAYER_SIZE, DENSE_LAYER_SIZE, 
# NUM_LSTM_LAYERS, NUM_DENSE_LAYERS, extra_features=["all_destinations"], var_time_len=VARIABLE_INPUT_LENGTH)

# save_path = "data/model_weights/checkpoints/cp_path_pred_kpi/in5out3/with_dest.pickle"

# # cp5 contains with none axis
# try:
#     # first do one epoch of training in order to initialize weights
#     # my_trainer.compile_and_fit(my_ds, 'data/model_weights/checkpoints/bin/test_cp1.ckpt', test_fit=True)
#     my_trainer.load_weights(save_path)
# except:
#     my_trainer.compile_and_fit(my_ds, save_path)

In [ ]:
# my_predictor = extended_predictor(g, my_trainer, 1)

# nxt_unsc, nxt_sc = my_ds.example_dict("test", "in_xy")

# output, output_s = my_trainer.predict_dict(nxt_unsc[0], False)

In [ ]:
# for i in range(1):
#     fig1, ax1 = plt.subplots()
#     scene_data.plot_on_image([nxt_sc[0]["in_xy"], nxt_sc[1], output_s], 
#     save_path='data/images/final_notebook/example_prediction.png', ms = [6, 1, 3], ax=ax1, 
#     colors=["green", "blue", "yellow"], axes_labels=["input", "label", "output"])
    
#     dest_locs = nxt_sc[0]["all_destinations"][i][:, 0:2]
#     dest_probs = nxt_sc[0]["all_destinations"][i][:, 2:3]
    
#     scene_data.plot_dest_probs(dest_locs, dest_probs, 2, 200,
#     ax = ax1)

In [ ]:
# # Calculate KPIs
# it_ds = iter(my_ds.tf_ds_dict["test"])

# for ds_entry in it_ds:
#     in_batch, output = ds_entry

#     prediction_batch, prediction_batch_scaled = my_trainer.predict_dict(in_batch, False)

#     kpi_val, c, nc = batch_kpi(avg_displacement_error, in_batch["labels"], prediction_batch_scaled, 2)



In [3]:
# train the networks on the available data
# included
n_in_future = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
n_in_future = [5]
percentage_of_path_avail = [.2, .3, .4, .5, .6, .7, .8, .9, 1.]
num_input_steps = [2,3,4,5,-1] #needs new network trained every time
num_output_steps = [3] #needs new network trained every time
dests_included = [True]
n_dense_layers = [2]
n_lstm_layers = [2]
dense_layer_sizes = [32, 128]
lstm_layer_sizes = [32, 128]
# not included

n_conn_points_incl = [False]
all_points_incl = [False]



In [ ]:
# my_trainer = DLTrainer(max_epochs=1, patience=PATIENCE)
# my_trainer.LSTM_one_shot_predictor_named_i(my_ds, LSTM_LAYER_SIZE, DENSE_LAYER_SIZE, 
# NUM_LSTM_LAYERS, NUM_DENSE_LAYERS, extra_features=["all_destinations", "n_connected_points_after"], var_time_len=VARIABLE_INPUT_LENGTH, size_dict=my_ds.size_dict)
# my_trainer.compile_and_fit2(my_ds.tf_ds_dict["train"], my_ds.tf_ds_dict["val"], save_path="save_path.pickle")

In [4]:

# set up the csv writer
csv_headers = ["n", "ADE", "FDE", "counter", "not_counter", "num_in_steps", "num_out_steps", "dests_included", "all_points_included", "n_conn_points_included", "num_lstm_layers", "num_dense_layers", "lstm_layer_size", "dense_layer_size"]
csv_folder_path = "data/results/destination_pred"
filename = 'results_' + datetime.now().strftime("%d_%m_%Y__%H_%M") + ".csv"
full_path = os.path.join(csv_folder_path, filename)

with open(full_path, 'w', newline='\n') as csvfile:
    my_writer = csv.writer(csvfile, delimiter=',')
    my_writer.writerow(csv_headers)

# Count entries in the big data set
# num_training_samples = 0
# for entry in iter(my_ds.tf_ds_dict["train"]):
#     num_training_samples+=1
# num_val_samples = 0
# for entry in iter(my_ds.tf_ds_dict["val"]):
#     num_val_samples+=1

for n_dense in n_dense_layers:  
    for n_lstm in n_lstm_layers:
        for dest_included in dests_included:
            for conn_point in n_conn_points_incl:   
                for all_points in all_points_incl:
                    for dense_layer_size in dense_layer_sizes:
                        for lstm_layer_size in lstm_layer_sizes:
                            ''' TRAIN THE MODEL '''
                            # # Get the reduced training sets
                            # smaller_train_ds = my_ds.tf_ds_dict["train"].take(math.ceil(num_training_samples*path_percentage))
                            # smaller_val_ds = my_ds.tf_ds_dict["val"].take(math.ceil(num_val_samples*path_percentage))

                            # Create the model instance
                            feat_list = []
                            if dest_included:
                                feat_list.append("all_destinations")
                            if conn_point:
                                feat_list.append("n_connected_points_after")
                            
                            my_trainer = DLTrainer(max_epochs=MAX_EPOCHS, patience=PATIENCE)
                            my_trainer.LSTM_one_shot_predictor_named_i(my_ds, LSTM_LAYER_SIZE, DENSE_LAYER_SIZE, 
                            n_lstm, n_dense, extra_features=feat_list, var_time_len=VARIABLE_INPUT_LENGTH, size_dict=my_ds.size_dict)

                            folder_path = "data/model_weights/checkpoints/cp_path_pred_kpi/%s" % (datetime.now().strftime("%d_%m_%Y__%H_%M"))
                            save_path = os.path.join(folder_path, "dest%sconnp%sallp%s.pickle" % (dest_included, conn_point, all_points))            
                            if not os.path.exists(folder_path):
                                os.mkdir(folder_path)
                            my_trainer.compile_and_fit2(my_ds.tf_ds_dict["train"], my_ds.tf_ds_dict["val"], save_path=save_path)

                            ''' GET KPIs FOR THIS MODEL '''   
                            print("Starting kpi calculation...")    
                            for n in tqdm(n_in_future):       
                                ade, fde, c, nc = tf_ds_kpi(my_ds.tf_ds_dict["test"], "labels", my_trainer, n, 
                                VARIABLE_INPUT_LENGTH)
                                
                                # Write them away for later analysis
                                line = [n, ade, fde, c, nc, SEQ_IN_LEN, SEQ_OUT_LEN, dest_included, all_points, conn_point, n_lstm, n_dense, lstm_layer_size, dense_layer_size]

                                with open(full_path, 'a', newline='\n') as csvfile:
                                    my_writer = csv.writer(csvfile, delimiter=',')
                                    my_writer.writerow(line)
                                

5
KerasTensor(type_spec=TensorSpec(shape=(None, 5, 2), dtype=tf.float32, name='in_xy'), name='in_xy', description="created by layer 'in_xy'")
KerasTensor(type_spec=TensorSpec(shape=(None, 64), dtype=tf.float32, name=None), name='flatten/Reshape:0', description="created by layer 'flatten'")
KerasTensor(type_spec=TensorSpec(shape=(None, 9, 3), dtype=tf.float32, name='all_destinations'), name='all_destinations', description="created by layer 'all_destinations'")
Epoch 1/100
1445/1445 [==============================] - 20s 11ms/step - loss: 0.0965 - mean_absolute_error: 0.2155 - val_loss: 0.0805 - val_mean_absolute_error: 0.2079

Epoch 00001: val_loss improved from inf to 0.08049, saving model to data/model_weights/checkpoints/cp_path_pred_kpi/20_07_2021__09_21/destTrueconnpFalseallpFalse.pickle
Alternative save happened on for epoch 0 happened on data/model_weights/checkpoints/cp_path_pred_kpi/20_07_2021__09_21/destTrueconnpFalseallpFalse.pickle for val_loss of 0.0805.
Epoch 2/100
1445/14

  0%|          | 0/1 [00:00<?, ?it/s]


Epoch 00025: val_loss did not improve from 0.02750
No alternative save happened on for epoch 24
Starting kpi calculation...


100%|██████████| 1/1 [00:24<00:00, 24.15s/it]


5
KerasTensor(type_spec=TensorSpec(shape=(None, 5, 2), dtype=tf.float32, name='in_xy'), name='in_xy', description="created by layer 'in_xy'")
KerasTensor(type_spec=TensorSpec(shape=(None, 64), dtype=tf.float32, name=None), name='flatten_2/Reshape:0', description="created by layer 'flatten_2'")
KerasTensor(type_spec=TensorSpec(shape=(None, 9, 3), dtype=tf.float32, name='all_destinations'), name='all_destinations', description="created by layer 'all_destinations'")
Epoch 1/100
1445/1445 [==============================] - 18s 10ms/step - loss: 0.0642 - mean_absolute_error: 0.1680 - val_loss: 0.0466 - val_mean_absolute_error: 0.1505

Epoch 00001: val_loss improved from inf to 0.04665, saving model to data/model_weights/checkpoints/cp_path_pred_kpi/20_07_2021__09_31/destTrueconnpFalseallpFalse.pickle
Alternative save happened on for epoch 0 happened on data/model_weights/checkpoints/cp_path_pred_kpi/20_07_2021__09_31/destTrueconnpFalseallpFalse.pickle for val_loss of 0.0466.
Epoch 2/100
144

...

100%|██████████| 1/1 [00:42<00:00, 42.72s/it]


5
KerasTensor(type_spec=TensorSpec(shape=(None, 5, 2), dtype=tf.float32, name='in_xy'), name='in_xy', description="created by layer 'in_xy'")
KerasTensor(type_spec=TensorSpec(shape=(None, 64), dtype=tf.float32, name=None), name='flatten_6/Reshape:0', description="created by layer 'flatten_6'")
KerasTensor(type_spec=TensorSpec(shape=(None, 9, 3), dtype=tf.float32, name='all_destinations'), name='all_destinations', description="created by layer 'all_destinations'")
Epoch 1/100
1445/1445 [==============================] - 28s 15ms/step - loss: 0.0676 - mean_absolute_error: 0.1705 - val_loss: 0.0621 - val_mean_absolute_error: 0.1737

Epoch 00001: val_loss improved from inf to 0.06209, saving model to data/model_weights/checkpoints/cp_path_pred_kpi/20_07_2021__09_50/destTrueconnpFalseallpFalse.pickle
Alternative save happened on for epoch 0 happened on data/model_weights/checkpoints/cp_path_pred_kpi/20_07_2021__09_50/destTrueconnpFalseallpFalse.pickle for val_loss of 0.0621.
Epoch 2/100
144

  0%|          | 0/1 [00:00<?, ?it/s]


Epoch 00015: val_loss did not improve from 0.02970
No alternative save happened on for epoch 14
Starting kpi calculation...


100%|██████████| 1/1 [00:25<00:00, 25.34s/it]


In [22]:
def calc_distances(a):
    return np.sqrt(np.square(a[0]-a[2])+np.square(a[1]-a[3]))

lala = np.array([[1., 0.], [2.,1.]])
lala = np.hstack([lala, lala+1])

np.apply_along_axis(f, 1, lala)


array([1.41421356, 1.41421356])